## TripAdvisor Recomender System

#### Data Collection from TripAdvisor

All the data are scraped using Selenium and BeautifulSoup from TripAdvisor website from late April to early May 2019. The hotels are from 16 popular European tourist destination cities and their TripAdvisor rating range from 3.0 to 5.0. The collected data are separated into 4 files. These files are:

1) tripadvisor_data.csv (hotel information from the hotel listings from the city section)

2) tripadvisor_hotel.txt (hotel information from the hotel details webpage in json format)

3) tripadvisor_reviewer.csv (brief reviewer information from the hotel details webpage)

4) tripadvisor_hotel_review.csv (reviews written by reviewer from the TripAdvisor reviewer webpage)

In [1]:
# import libraries  
from selenium import webdriver
from bs4 import BeautifulSoup 
import urllib
import requests, re
from datetime import datetime
import time as t
from time import time
import os, sys
from lxml import html, etree
import argparse

import pandas as pd
import numpy as np
import json
from collections import defaultdict

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore,  BaselineOnly, CoClustering
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise import dump
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

### Scraping through TripAdvisor City Section for Hotel Listings

In [ ]:

def process_hotel_list(locality, checkin_date, checkout_date, hotel_data, sort="popularity"):
    
    """
    
    Args:
        locality is the city location of the hotel search

        checkin_date and checkout_date are the start and end date of hotel stay.
        It should be year/month/day string format, ie. '2019/07/01'

        sort is the method of how search results are displayed either by order of 'popularity', 'value', 'price' or 'distance'

        hotel_data is a list used to collect scraped hotel information which will be returned back by the function.
    
    """
    
    check_in = checkin_date
    check_out = checkout_date
    
    geo_url = 'https://www.tripadvisor.com/TypeAheadJson?action=API&startTime='+str(int(time()))+'&uiOrigin=GEOSCOPE&source=GEOSCOPE&interleaved=true&types=geo,theme_park&neighborhood_geos=true&link_type=hotel&details=true&max=12&injectNeighborhoods=true&query='+locality
    api_response  = requests.get(geo_url, verify=False).json()
    url_from_autocomplete = "http://www.tripadvisor.com"+api_response['results'][0]['url']
    geo = api_response['results'][0]['value'] 
    date = check_in+"_"+check_out
    
    form_data = {'changeSet': 'TRAVEL_INFO',
                'showSnippets': 'false',
                'staydates':date,
                'uguests': '2',
                'sortOrder':sort
    
                }
    
    headers = {
                'Accept': 'text/javascript, text/html, application/xml, text/xml, */*',
                'Accept-Encoding': 'gzip,deflate',
                'Accept-Language': 'en-US,en;q=0.5',
                'Cache-Control': 'no-cache',
                'Connection': 'keep-alive',
                'Content-Type': 'application/x-www-form-urlencoded; charset=utf-8',
                'Host': 'www.tripadvisor.com',
                'Pragma': 'no-cache',
                'Referer': url_from_autocomplete,
                'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:28.0) Gecko/20100101 Firefox/28.0',
                'X-Requested-With': 'XMLHttpRequest'
                }
    
    cookies=  {"SetCurrency":"USD"}
    page_response  = requests.post(url = url_from_autocomplete,data=form_data,headers = headers, cookies = cookies, verify=False)
    if page_response.status_code != 200:
        print('Page error for, '+ url_from_autocomplete +', is found.')
        return hotel_data
    else:
        parser = html.fromstring(page_response.text)
        hotel_data = iterate_hotels(parser, locality, hotel_data)
        return hotel_data
    
    
    
def iterate_hotels(parser, locality, hotel_data):
    
    """
    
    this function is called by process_hotel_list() in order to parse and scrape hotel information
    
    """
    
    next_page = parser.xpath('//a[contains(@class,"nav next taLnk ui_button primary")]/@href')
    next_page = 'http://www.tripadvisor.com' + next_page[0].replace('#BODYCON','') if next_page else  None
    print("Processing content from " + next_page)
    
    hotel_lists = parser.xpath('//div[contains(@class,"listItem")]//div[contains(@class,"listing collapsed")]')
    if not hotel_lists:
        hotel_lists = parser.xpath('//div[contains(@class,"listItem")]//div[@class="listing "]')

    for index, hotel in enumerate(hotel_lists): # grab the element of each feature from html
        XPATH_HOTEL_LINK = './/a[contains(@class,"property_title")]/@href'
        XPATH_REVIEWS  = './/a[@class="review_count"]//text()'
        XPATH_RANK = './/div[@class="popRanking"]//text()'
        XPATH_RATING = './/a[contains(@class,"ui_bubble_rating")]//@alt'
        XPATH_HOTEL_NAME = './/a[contains(@class,"property_title")]//text()'
        XPATH_HOTEL_FEATURES = './/div[contains(@class,"common_hotel_icons_list")]//li//text()'
        XPATH_HOTEL_PRICE = './/div[contains(@data-sizegroup,"mini-meta-price")]/text()'
        XPATH_VIEW_DEALS = './/div[contains(@data-ajax-preserve,"viewDeals")]//text()' 
        XPATH_BOOKING_PROVIDER = './/div[contains(@data-sizegroup,"mini-meta-provider")]//text()'

        raw_booking_provider = hotel.xpath(XPATH_BOOKING_PROVIDER)
        raw_no_of_deals =  hotel.xpath(XPATH_VIEW_DEALS)
        raw_hotel_link = hotel.xpath(XPATH_HOTEL_LINK)
        raw_no_of_reviews = hotel.xpath(XPATH_REVIEWS)
        raw_rank = hotel.xpath(XPATH_RANK)
        raw_rating = hotel.xpath(XPATH_RATING)
        raw_hotel_name = hotel.xpath(XPATH_HOTEL_NAME)
        raw_hotel_features = hotel.xpath(XPATH_HOTEL_FEATURES)
        raw_hotel_price_per_night  = hotel.xpath(XPATH_HOTEL_PRICE)

        url = 'http://www.tripadvisor.com'+raw_hotel_link[0] if raw_hotel_link else  None
        reviews = ''.join(raw_no_of_reviews).replace("reviews","").replace(",","") if raw_no_of_reviews else 0 
        rank = ''.join(raw_rank) if raw_rank else None
        rating = ''.join(raw_rating).replace('of 5 bubbles','').strip() if raw_rating else None
        name = ''.join(raw_hotel_name).strip() if raw_hotel_name else None
        hotel_features = ','.join(raw_hotel_features)
        price_per_night = ''.join(str(*raw_hotel_price_per_night)).replace('\n','') if raw_hotel_price_per_night else None
        no_of_deals = re.findall("all\s+?(\d+)\s+?",''.join(raw_no_of_deals))
        booking_provider = ''.join(raw_booking_provider).strip() if raw_booking_provider else None

        if no_of_deals:
            no_of_deals = no_of_deals[0]
        else:
            no_of_deals = 0

        data = {
                'hotel_name':name,
                'url':url,
                'locality':locality,
                'reviews':reviews,
                'tripadvisor_rating':rating,
                'hotel_features':hotel_features,
                'price_per_night':price_per_night,
                'no_of_deals':no_of_deals,
                'booking_provider':booking_provider

                }
        hotel_data.append(data)
        
    if next_page:
        t.sleep(1)
    try:
        page_response = requests.get(next_page)
    except requests.exceptions.ConnectionError:
        t.sleep(60)
        page_response = requests.get(next_page)
        parser = html.fromstring(page_response.text)
        hotel_data = iterate_hotels(parser, locality, hotel_data)
    else:
        parser = html.fromstring(page_response.text)
        hotel_data = iterate_hotels(parser, locality, hotel_data)
    finally:
        return hotel_data
        
    return hotel_data


In [ ]:
hotel_data = []

In [ ]:
hotel_data = process_hotel_list('London, England', '2019/07/01', '2019/07/15', hotel_data)

In [ ]:
# write hotel_data to a csv file called tripadvisor_data.csv
# with open('tripadvisor_data.csv','wb')as csvfile:  # substitute 'wb' for 'ab' to append hotel_data to existing tripadvisor_data.csv
            fieldnames = ['hotel_name','url','locality','reviews','tripadvisor_rating','price_per_night','booking_provider','no_of_deals','hotel_features']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for row in hotel_data:
                writer.writerow(row)

In [ ]:
tadata = pd.read_csv('tripadvisor_data.csv')
tadata.info()

In [ ]:
tadata = tadata.drop_duplicates() # drop duplicate hotels from list
# tadata.to_csv("tripadvisor_data.csv", index=False, encoding='utf8')

### Scraping through Individual TripAdvisor Hotel Webpage for Hotel Details

In [ ]:

def process_hotel_details(hotel_url, retry=0):
    
    """
    
    hotel_url is hotel webpage at TripAdvisor
    retry attempts to reconnect to TripAdvisor if it fails up to MAX_RETRY times
    
    """
    
    MAX_RETRY = 10
    RETRY = 0

    headers = {
                "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
                "accept-encoding": "gzip, deflate, br",
                "accept-language": "en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7",
                "cache-control": "max-age=0",
                "upgrade-insecure-requests": "1",
                "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36",
                }

    response = requests.get(hotel_url, headers=headers)
    if response.status_code == 404:
        print('error: Page not found, status_code: 404')
        pass
    parser = html.fromstring(response.text, myurl)

    XPATH_NAME = '//h1[@id="HEADING"]//text()'
    XPATH_RANK = '//span[contains(@class,"popularity")]//text()'
    XPATH_FULL_ADDRESS_JSON = '//script[@type="application/ld+json"]//text()'

    raw_name = parser.xpath(XPATH_NAME)
    raw_rank = parser.xpath(XPATH_RANK)
    raw_address_json = parser.xpath(XPATH_FULL_ADDRESS_JSON)
    name = clean(raw_name)
    rank = clean(raw_rank)
    if not name:
        if RETRY < MAX_RETRY:
            RETRY = RETRY+1
            # Retrying the same URL
            process_hotel_details(hotel_url, RETRY)

    hotel_rating = 0
    review_count = 0
    address = {}
    if raw_address_json:
        try:
            parsed_address_info = json.loads(raw_address_json[0])
            rating = parsed_address_info.get('aggregateRating', {})
            address = parsed_address_info.get("address", {})

            hotel_rating = rating.get('ratingValue')
            review_count = rating.get('reviewCount')

            address = {
                        'street_address': address.get('streetAddress'),
                        'region': address.get('addressRegion'),
                        'locality': address.get('addressLocality'),
                        'country': address.get("addressCountry", {}).get("name"),
                        'zipcode': address.get("postalCode")
                    }
        except Exception as e:
            review_count = hotel_rating = 0
            raise e

    ratings = {}
    elems = parser.find_class("hotels-review-list-parts-ReviewRatingFilter__row_num--gIW_f")
    if elems:
        ratings = {
            'Excellent': elems[0].text,
            'Good': elems[1].text,
            'Average': elems[2].text,
            'Poor': elems[3].text,
            'Terrible': elems[4].text
                }

    amenities = parser.find_class("hotels-hotel-review-about-with-photos-Amenity__name--2IUMR")
    amenity_list = []     
    for a in amenities:
        amenity_list.append(a.text)

    data = {
            'address': address,
            'ratings': ratings,
            'amenities': amenity_list,
            'rating': float(hotel_rating) if hotel_rating else 0.0,
            'review_count': int(review_count) if review_count else 0,
            'name': name,
            'rank': rank,
            'hotel_url': hotel_url
            }

    return data


def clean(text):
    
    """
    
    processing scraped information before adding to a json formatted list of dictionaries
    
    """
    
    if text:
        # Removing \n \r and \t
        return ' '.join(''.join(text).split()).strip()
    return None


In [ ]:
hotel_details = []
for index, hotel in enumerate(tadata['url'][0:1000]):
    result = process_request(hotel)
    hotel_details.append(result)
    print(index)  # keeping track of progress in case of error
    t.sleep(1)

In [ ]:
details = []
details.extend(hotel_details)

In [ ]:
# saving details list as tripadvisor_hotel.txt in json format
# with open('tripadvisor_hotel.txt', 'w') as out_file:
    json.dump(details, out_file)

In [ ]:
# converting tripadvisor_hotel.txt to pandas DataFrame
tahotel = pd.read_json('tripadvisor_hotel.txt')
tahotel = tahotel[tahotel.review_count >= 100]   # filtering hotels with more than 100 reviews 
tahotel.index = range(len(tahotel))

### Scraping through Individual TripAdvisor Hotel Webpage for Guest Review Information

In [ ]:

def process_reviews(hotel_url, reviewer_list):
    
    response = requests.get(hotel_url)
    if response.status_code != 200:
        print('Error is found on page, '+ hotel_url)
        return reviewer_list
    else:
        soup = BeautifulSoup(response.text)
        reviews = soup.findAll(class_ = 'hotels-review-list-parts-SingleReview__reviewContainer--d54T4')
        review_count = 10
        reviewer_list = iterate_review_page(soup, reviews, review_count, reviewer_list)
        return reviewer_list
            
def iterate_review_page(soup, reviews, review_count, reviewer_list):           

    next_page = soup.find('a', class_="ui_button nav next primary ", href=True)
    next_page = 'http://www.tripadvisor.com'+next_page['href'] if next_page['href'] else  None
       
    for r in reviews:
        try:
            contribute = r.find('span', class_ = 'social-member-MemberHeaderStats__bold--3z3qh').get_text()
            contribute = int(contribute.replace(',', '')) 
        except AttributeError:
            contribute = 0    
            pass
        
        if contribute > 25 and review_count > 0: #filter reviewer with more than 25 reviews and a limit of 10 reviewers for each hotel
            try:
                reviewer = r.find('a', class_ = 'ui_header_link social-member-event-MemberEventOnObjectBlock__member--35-jC').get_text()
            except AttributeError:
                reviewer = "NONE PROVIDED"
                
            reviewer_website = r.find('a', href=True)
            reviewer_website = 'http://www.tripadvisor.com'+reviewer_website['href']
            rating = r.find('span', class_="ui_bubble_rating")
            rating = str(rating).strip('"0></span>')[-1:]
            try:
                date =  r.find(class_ = 'hotels-review-list-parts-EventDate__event_date--CRXs4').find('span').get_text()
            except:
                date = ''
            
            if reviewer_website not in list(reviewer_list['user_link']): # avoid duplicate reviewer
                
                df = pd.DataFrame({'uid':reviewer, 'user_link':reviewer_website, 'rating':int(rating), 'date_of_stay':date[14:], 'num_of_reviews':int(contribute)}, 
                                  columns=['uid', 'user_link', 'rating', 'date_of_stay', 'num_of_reviews'], index=[0])
                reviewer_list = pd.concat([reviewer_list, df], axis =0)   
                review_count -= 1
            else:
                pass
    if next_page and review_count > 0:
        t.sleep(1)
        try:
            response = requests.get(next_page)
        except requests.exceptions.ConnectionError:
            t.sleep(60) # Retrying same url after 1 minute
            response = requests.get(next_page)
            soup = BeautifulSoup(response.text)
            reviews = soup.findAll(class_ = 'hotels-review-list-parts-SingleReview__reviewContainer--d54T4')
            reviewer_list = iterate_review_page(soup, reviews, review_count, reviewer_list)
        else:
            soup = BeautifulSoup(response.text)
            reviews = soup.findAll(class_ = 'hotels-review-list-parts-SingleReview__reviewContainer--d54T4')
            reviewer_list = iterate_review_page(soup, reviews, review_count, reviewer_list)
        finally:
            return reviewer_list

    return reviewer_list



In [ ]:
for index, hotel in enumerate(tahotel[0:1000]['hotel_url']):
    hotel_url = hotel
    print(index) # keeping track of progress in case of error
    n = len(reviewer_list)
    hotel_df = process_reviews(hotel_url, reviewer_list)
    reviewer_list = pd.concat([reviewer_list, hotel_df[:][n:]], axis =0)
    t.sleep(1)

In [ ]:
reviewer_list.to_csv('tripadvisor_reviewer.csv', index=False)

### Scraping through TripAdvisor Guest Reviewer Webpage for Hotel Reviews using Selenium

In [2]:

def process_other_reviews(user_url, hotel_review_list):
    
    driver.get(user_url)
    t.sleep(1.5)
    try: # check if driver get user_url page
        response = driver.page_source
        hotel_review_list = try_button(driver, response, user_url, hotel_review_list)   
    except: # retry getting user_url with Selenium
        driver.get(user_url)
        t.sleep(1.5)
        try:
            response = driver.page_source
        except Exception as e: # print reason for why it fails
            print(e)
        else: # check if 'show more' button is clickable
            hotel_review_list = try_button(driver, response, user_url, hotel_review_list)
            
    return hotel_review_list
 

def try_button(driver, response, user_url, hotel_review_list):   # check if 'show more' button is clickable
    
    try:
        python_button = driver.find_element_by_class_name("social-show-more-ShowMore__button_contents--1djai")
        python_button.click()
        soup = BeautifulSoup(response)
        hotel_review_list = iterate_reviewer_page(soup, user_url, hotel_review_list)
    except:
        soup = BeautifulSoup(response)
        hotel_review_list = iterate_reviewer_page(soup, user_url, hotel_review_list)
        
    return hotel_review_list

    
def iterate_reviewer_page(soup, user_url, hotel_review_list):    # filter out reviews that are not of hotels from the 16 european cities 
    
    reviews = soup.findAll('div', class_ = 'social-sections-CardSection__card_section--3Hc9Y ui_card section')
    for r in reviews:
        hotel = r.find('div', class_="social-sections-POICarousel__container--297jy social-sections-POICarousel__carousel--1vz03").find('a', href=True)
        if not hotel:
            continue
        try:
            hotel = hotel['href']    # grab website link info
        except:
            continue
        if "Hotel_Review" in hotel:    # check if it is a hotel review and from the 16 cities
            if 'London_England.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            elif 'Paris_lle_de_France.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            elif 'Rome_Lazio.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            elif 'Barcelona_Catalonia.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            elif 'Berlin.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            elif 'Vienna.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            elif 'Prague_Bohemia.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            elif 'Budapest_Central_Hungary.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            elif 'Athens_Attica.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            elif 'Florence_Tuscany.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            elif 'Milan_Lombardy.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            elif 'Madrid.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            elif 'Lisbon_District_Central_Portugal.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            elif 'Edinburgh_Scotland.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            elif 'Amsterdam_North_Holland_Province.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            elif 'Brussels.html' in hotel:
                hotel_review_list = scrape_reviews(r, user_url, hotel_review_list, hotel)
            else:
                pass
            
    return hotel_review_list


def scrape_reviews(r, user_url, hotel_review_list, hotel):
    
    reviewer = r.find('a', class_ = 'ui_link social-member-event-MemberEventOnObjectBlock__member--35-jC').get_text()
    try:
        title = r.find('div', class_="social-sections-ReviewSection__title--35ISZ social-sections-ReviewSection__linked--2rTun").get_text()
    except:
        title = ''
    try:
        preview = r.find('q', class_="social-sections-ReviewSection__quote--3gE7d").get_text()
    except:
        preview = ''
    rating = r.find('span', class_="ui_bubble_rating")
    rating = str(rating).strip('"0></span>')[-1:]
    hotel = 'http://www.tripadvisor.com' + hotel
    try:
        date =  r.find('div', class_ = 'social-review-info-EventDate__event_date--2d3vn').find('span').get_text()
    except:
        date = ''
    df = pd.DataFrame({'uid':reviewer, 'user_link':user_url, 'rating':float(rating), 'hotel_link':hotel, 'date_of_stay':date[14:], 'title':title, 'review_preview':preview}, 
                                  columns=['uid', 'user_link', 'rating', 'hotel_link', 'date_of_stay', 'title', 'review_preview'], index=[0])
    hotel_review_list = pd.concat([hotel_review_list, df])   
    hotel_review_list = hotel_review_list.reset_index(drop=True)
    return hotel_review_list


In [3]:
hotel_review_list = pd.DataFrame(columns=['uid', 'user_link', 'rating', 'hotel_link', 'date_of_stay', 'title', 'review_preview'])    

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument("--test-type")
driver = webdriver.Chrome(chrome_options=options)
# driver = webdriver.Chrome("chromedriver")

/Users/tzulungs/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [7]:
for index, user_url in enumerate(reviewer_list.user_link[0:10000]):
    print(index)
    hotel_review_list = process_other_reviews(user_url, hotel_review_list)

0
1
2
3
4
5
6
7
8
9
10
11
12


KeyboardInterrupt: 

### Data Cleaning And Preliminary Data Evaluation

In [ ]:
tadata = pd.read_csv('tripadvisor_data.csv')

In [ ]:
# cleaning 'reviews' column and reassign column to integer type
tadata['reviews'].replace('1 review', '1', inplace=True)
tadata['reviews'] = tadata.reviews.astype('int64')

# dropping 'tripadvisor_rating' that are less than 3.0
tadata = tadata.drop(tadata[tadata['tripadvisor_rating'] < 3.0].index)

# cleaning 'price_per_night' column and reassign column to integer type
tadata['price_per_night'] = tadata['price_per_night'].apply(lambda x: str(x).strip('$'))
tadata['price_per_night'] = tadata['price_per_night'].apply(lambda x: str(x).replace(',', ''))
tadata = tadata.drop(tadata.loc[tadata['price_per_night']=='nan'].index)
tadata['price_per_night'] = tadata.price_per_night.astype('int64')

print(tadata.info())
print(tadata.describe())

There are a few nan values in tripadvisor_rating. Since having a rating is necessary for the recommender system, the rows with the missing rating will be dropped. The distribution of 'price_per_night' is skewed and taking a log function is warranted.

In [ ]:
tadata = tadata[np.isfinite(tadata['tripadvisor_rating'])]
tadata['log_ppp'] = tadata.price_per_night.apply(lambda x: np.log(x))
tadata.head()

In [ ]:
sns.set(rc={'figure.figsize':(15, 8)})
box_fig = sns.boxplot(x="locality", y="log_ppp", data=tadata)
box_fig.set_xticklabels(tadata.locality.unique(), rotation=70)

In [ ]:
tadata1 = tadata.groupby(['locality', 'tripadvisor_rating'])
grouped1 = tadata1['price_per_night'].agg(np.mean).round(2)
grouped1 = grouped1.unstack(level=-1)
grouped1

In [ ]:
group_barplot1 = sns.catplot(x="locality", y="log_ppp", hue="tripadvisor_rating", data=tadata,
                height=6, kind="bar", size=5, aspect=4, palette="muted")
group_barplot1.despine(left=True)
group_barplot1.set_ylabels("log value of price_per_night")
group_barplot1.set_xticklabels(tadata.locality.unique(), rotation=70)


In [ ]:
grouped2 = tadata1['hotel_name'].count()
grouped2 = grouped2.unstack(level=-1)
grouped2

In [ ]:
grouped2.plot.bar(figsize=(15, 6), cmap=plt.cm.rainbow)

In [ ]:
tadata2 = tadata.groupby(["tripadvisor_rating", pd.cut(tadata["no_of_deals"], np.arange(0, 25, 5))])
grouped3 = tadata2['price_per_night'].agg(np.mean).round(2)
grouped3 = grouped3.unstack(level=1)
print(grouped3)
grouped3.plot.bar(figsize=(10, 6), cmap=plt.cm.rainbow)

In [ ]:
tahotel = pd.read_json('tripadvisor_hotel.json')
tahotel.head()

In [ ]:
#cleaning tripadvisor_hotel.json
tahotel = pd.concat([tahotel.drop(['address'], axis=1), tahotel['address'].apply(pd.Series)], axis=1)
tahotel.drop('region', axis=1, inplace=True)

# make new columns from the dictionary of values in the ratings column and converting it to integers
tahotel = pd.concat([tahotel.drop(['ratings'], axis=1), tahotel['ratings'].apply(pd.Series)], axis=1)
tahotel['Excellent'] = tahotel['Excellent'].apply(lambda x: str(x).replace(',', ''))
tahotel['Good'] = tahotel['Good'].apply(lambda x: str(x).replace(',', ''))
tahotel['Average'] = tahotel['Average'].apply(lambda x: str(x).replace(',', ''))
tahotel['Poor'] = tahotel['Poor'].apply(lambda x: str(x).replace(',', ''))
tahotel['Terrible'] = tahotel['Terrible'].apply(lambda x: str(x).replace(',', ''))

tahotel['Excellent'] = tahotel.Excellent.astype('int64', errors='ignore')
tahotel['Good'] = tahotel.Good.astype('int64', errors='ignore')
tahotel['Average']= tahotel.Average.astype('int64', errors='ignore')
tahotel['Poor'] = tahotel.Poor.astype('int64', errors='ignore')
tahotel['Terrible'] = tahotel.Terrible.astype('int64', errors='ignore')

# get dummy variables from amenities column
tahotel = pd.concat([tahotel.drop('amenities', axis=1), pd.get_dummies(tahotel['amenities'].apply(pd.Series).stack()).sum(level=0)], axis=1)

tahotel.head()

In [ ]:
combined_hotel_reviews = pd.read_csv('tripadvisor_hotel_review.csv')
combined_hotel_reviews.head()

In [ ]:
# create unique ids for hotels and guest reviewers
combined_hotel_reviews = combined_hotel_reviews.assign(uid=(combined_hotel_reviews['user_link']).astype('category').cat.codes)
combined_hotel_reviews = combined_hotel_reviews.assign(hid=(combined_hotel_reviews['hotel_link']).astype('category').cat.codes)
# dropping off 3 columns
combined_hotel_reviews.drop(combined_hotel_reviews[['date_of_stay', 'title', 'review_preview']], axis=1, inplace=True)
combined_hotel_reviews.head()

In [ ]:
combined_hotel_reviews['hotel_link'].isin(tahotel['hotel_url']).value_counts()

In [ ]:
new_hotels = combined_hotel_reviews[~combined_hotel_reviews['hotel_link'].isin(tahotel['hotel_url']).dropna()]
len(new_hotels)

In [ ]:
combined_hotel_reviews = combined_hotel_reviews.drop(combined_hotel_reviews.index[13328])

In [ ]:
combined_hotel_reviews = combined_hotel_reviews.reset_index(drop=True)

In [ ]:
combined_hotel_reviews = pd.merge(combined_hotel_reviews,
                 tahotel[['hotel_url', 'name', 'street_address', 'locality', 'country', 'hotel_rating']],
                 left_on='hotel_link',
                 right_on='hotel_url',
                 how='left')
combined_hotel_reviews = combined_hotel_reviews.drop(combined_hotel_reviews[['hotel_url']], axis=1)
combined_hotel_reviews.head()

## Recommender System with Surprise

In [ ]:
grouped_combined_reviews = combined_hotel_reviews.groupby(['hid', 'uid'])
grouped_rating = grouped_combined_reviews[['rating']].agg(np.mean).round(2)
grouped_rating.reset_index(inplace=True)
grouped_rating.head()

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(grouped_rating[['uid', 'hid', 'rating']], reader)

In [ ]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse') 

In [ ]:
grid_search_result = pd.DataFrame(columns=['algorithm', 'param_grid', 'best_rmse_score', 'best_params', 'test_prediction_rmse', 'fit_time'])

In [ ]:
trainset, testset = train_test_split(data, test_size=.25)

In [ ]:
print('Using SVDpp')

n_epochs = [15, 20, 25]
lr_all = [0.005, 0.01, 0.015]
reg_all = [0.3, 0.4, 0.5]

param_grid = {'n_epochs': n_epochs, 'lr_all': lr_all, 'reg_all': reg_all}
gs1 = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=5)
# t0 = t.time()
gs1.fit(data)
# t1 = t.time()
# lapsed = t1-t0

print(gs1.best_score['rmse'])
print(gs1.best_params['rmse'])

algo = gs1.best_estimator['rmse']
predictions1 = algo.fit(trainset).test(testset)
accuracy.rmse(predictions1)

# grid_search_result = grid_search_result.append({'algorithm':'SVDpp', 'param_grid':{'n_epochs':n_epochs, 'lr_all':lr_all, 'reg_all':reg_all}, 'best_rmse_score':gs1.best_score['rmse'], 'best_params':gs1.best_params['rmse'], 'test_prediction_rmse':accuracy.rmse(predictions1), 'fit_time':lapsed}, ignore_index=True)

In [ ]:
print('KNN Baseline Using ALS and SGD')

method = ['als', 'sgd']
reg = [1, 2]
k = [2, 3]
name = ['msd', 'cosine']
min_support = [1, 5, 10]
user_based = [False]

param_grid = {'bsl_options': {'method':method, 'reg':reg}, 'k':k, 'sim_options': {'name':name, 'min_support':min_support, 'user_based':user_based}}
gs2 = GridSearchCV(KNNBaseline, param_grid, measures=['rmse', 'mae'], cv=5)
t0 = t.time()
gs2.fit(data)
t1 = t.time()
lapsed = t1-t0

print(gs2.best_score['rmse'])
print(gs2.best_params['rmse'])

algo = gs2.best_estimator['rmse']
predictions2 = algo.fit(trainset).test(testset)
accuracy.rmse(predictions2)

# grid_search_result = grid_search_result.append({'algorithm':'KNNBaseline', 'param_grid':{'method':method, 'reg':reg}, 'k':k, 'sim_options': {'name':name, 'min_support':min_support, 'user_based':user_based}, 'best_rmse_score':gs2.best_score['rmse'], 'best_params':gs2.best_params['rmse'], 'test_prediction_rmse':accuracy.rmse(predictions2), 'fit_time':lapsed}, ignore_index=True)

In [ ]:
print('Using SVD')

n_epochs = [20, 30, 40]
lr_all = [0.005, 0.01, 0.02]
reg_all = [0.3, 0.4, 0.5]

param_grid = {'n_epochs':n_epochs, 'lr_all':lr_all, 'reg_all':reg_all}
gs3 = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
t0 = t.time()
gs3.fit(data)
t1 = t.time()
lapsed = t1-t0

print(gs3.best_score['rmse'])
print(gs3.best_params['rmse'])

algo = gs3.best_estimator['rmse']
predictions3 = algo.fit(trainset).test(testset)
accuracy.rmse(predictions3)

# grid_search_result = grid_search_result.append({'algorithm':'SVD', 'param_grid':{'n_epochs':n_epochs, 'lr_all':lr_all, 'reg_all':reg_all}, 'best_rmse_score':gs3.best_score['rmse'], 'best_params':gs3.best_params['rmse'], 'test_prediction_rmse':accuracy.rmse(predictions3), 'fit_time':lapsed}, ignore_index=True)

In [ ]:
print('Baseline Only Using ALS and SGD')

method = ['als', 'sgd']
reg = [1, 2]
k = [1, 2, 3]

param_grid = {'bsl_options': {'method':method, 'reg':reg, 'k': k}}
gs3 = GridSearchCV(BaselineOnly, param_grid, measures=['rmse', 'mae'], cv=5)
t0 = t.time()
gs3.fit(data)
t1 = t.time()
lapsed = t1-t0

print(gs3.best_score['rmse'])
print(gs3.best_params['rmse'])

algo = gs3.best_estimator['rmse']
predictions3 = algo.fit(trainset).test(testset)
accuracy.rmse(predictions3)

# grid_search_result = grid_search_result.append({'algorithm':'BaselineOnly', 'param_grid':{'method':method, 'reg':reg, 'k': k}, 'best_rmse_score':gs3.best_score['rmse'], 'best_params':gs3.best_params['rmse'], 'test_prediction_rmse':accuracy.rmse(predictions3), 'fit_time':lapsed}, ignore_index=True)

In [ ]:
grid_search_result.head()

In [ ]:
# Dump best algorithm and reload it.
file_name = os.path.expanduser('surprise_SVD_model')
dump.dump(file_name, algo=algo)
_, loaded_algo = dump.load(file_name)


In [ ]:
def get_top_n(predictions, n=10):
    
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


In [ ]:
# First train an SVD algorithm
data = Dataset.load_from_df(grouped_rating[['uid', 'hid', 'rating']], reader)
trainset = data.build_full_trainset()
algo = loaded_algo
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [ ]:
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])